In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
#Load DataSets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_out = pd.read_csv("test.csv")
test['SalePrice'] = 0

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
def data_clean(ds,test):
    #print(ds.head())
    ds['MSZoning'].replace({'C (all)': 0, 'RM':1, 'RH':2, 'RL':3, 'FV':4}, inplace = True) 
    numeric_features = ds.select_dtypes(include=[np.number])
    corr = numeric_features.corr()
    #print (corr['SalePrice'].sort_values(ascending=False))
    a = numeric_features.corr()['SalePrice'].sort_values(ascending=False)[:10] #pega as 10 maiores correlações
    print(a)
    index = a.index #retira o index delas 
    ds[index] = ds[index].fillna(0)
    ds = ds[index]
    ds = ds.dropna()
    
    test['MSZoning'].replace({'C (all)': 0, 'RM':1, 'RH':2, 'RL':3, 'FV':4}, inplace = True)
    test[index] = test[index].fillna(0)
    test = test[index]
    test = test.dropna()
    #print(ds.isnull().sum())
    
    return ds,test

In [ ]:
def not_numeric_converter(train,test):
    ds = train
    ds_test = test
    ds_not_num = pd.DataFrame() #Create new data frame to not numbers
    numeric_features = ds.select_dtypes(include=[np.number]) #data frame with only numeric features
    #numeric_features.drop('SalePrice',axis='columns', inplace=True)
    numeric_features = numeric_features.columns #separate only index
    ds_not_num = ds #copy ds to ds_not_num
    ds_not_num.drop(numeric_features,axis='columns', inplace=True) #drop all numeric in ds_not_num #ERROR, está dropando todos os data frames principais 
    print(train)
    print(ds)
    for i in ds_not_num.columns:
        print(i)
        a = ds[[i, 'SalePrice']].groupby(i).mean()
        #print(a['SalePrice'].sort_values(ascending=True))
        index = ds[[i]].groupby(i).mean().index #extract all results in column as index
        count = 1
        for z in index:
            ds[i].replace({z: count}, inplace = True)
            ds_test[i].replace({z: count}, inplace = True)
            count = count + 1
            
        
    return(ds,ds_test)

In [ ]:
#(a,b) = not_numeric_converter(train,test)

In [ ]:
train.head()

In [ ]:
'''
numeric_features = test.select_dtypes(include=[np.number])
numeric_features = numeric_features.columns
ds_not_num = pd.DataFrame()
test.drop(numeric_features,axis='columns', inplace=True)
'''

In [ ]:
for i in test.columns:
    a = train[[i, 'SalePrice']].groupby(i).mean()
    #print(a['SalePrice'].sort_values(ascending=True))
    index = train[[i]].groupby(i).mean().index #extract all results in column as index
    count = 1
    for z in index:
        test[i].replace({z: count}, inplace = True)
        count = count + 1
        

In [ ]:
train.select_dtypes(include=[np.number]).columns
#train.columns

In [6]:
###Teste
(train,test) = data_clean(train,test)
#test = data_clean(test)

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
Name: SalePrice, dtype: float64


In [3]:
train.groupby('GarageCars').mean()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
GarageCars,,,,,,,,,,,,,,,,,,,,,
0,707.012346,71.790123,60.405405,7469.679012,4.604938,5.530864,1942.481481,1975.172840,13.777778,240.382716,...,39.555556,36.172840,37.530864,0.000000,0.000000,0.000000,53.703704,6.111111,2007.876543,103317.283951
1,725.200542,50.230352,62.548077,8636.685637,5.189702,5.964770,1948.243902,1971.257453,39.753388,333.230352,...,65.479675,23.449864,38.750678,0.867209,13.078591,1.560976,81.165312,6.265583,2007.850949,128116.688347
2,734.627427,60.934466,70.624611,11076.458738,6.251214,5.510922,1978.787621,1988.411408,100.756426,468.156553,...,98.776699,51.533981,14.899272,5.283981,17.461165,3.515777,34.750000,6.291262,2007.836165,183851.663835
3,725.292818,44.475138,85.798817,13063.127072,7.950276,5.104972,1997.287293,2000.850829,287.394444,643.933702,...,155.906077,77.773481,11.331492,1.679558,15.331492,3.066298,3.093923,6.662983,2007.618785,309636.121547
4,1010.400000,92.000000,76.000000,14232.000000,5.400000,5.200000,1955.600000,1983.200000,143.400000,593.600000,...,124.000000,0.000000,77.200000,0.000000,0.000000,0.000000,0.000000,6.600000,2008.000000,192655.800000


In [5]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [ ]:
X = train.drop(['SalePrice'], axis='columns')
y = train['SalePrice']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#Random Forest Classifier
model2=RandomForestClassifier(n_estimators =100)
model2.fit(X_train,y_train)
pred_2=model2.predict(X_test)
rcf=accuracy_score(pred_2,y_test);
print(rcf)   

In [ ]:
#Logistic Regression
model1 = LogisticRegression(solver='lbfgs', max_iter=1000)
model1.fit(X_train, y_train)
pred = model1.predict(X_test)
log_acc = accuracy_score(pred, y_test)
print(log_acc)

In [ ]:
#index = index.drop('SalePrice')

In [ ]:
test.drop('SalePrice',axis = 'columns', inplace = True)

In [ ]:
model2.fit(X,y)
pred = model2.predict(test)

In [ ]:
test.isnull().sum()

In [ ]:
result = pd.DataFrame()
result['Id'] = test_out.Id
result['SalePrice'] = pred

In [ ]:
result.head(5)

In [ ]:
result.to_csv('sub.csv', index = False)

In [ ]:
ds = train
ds_test = test
train.head()

In [ ]:
ds_not_num = pd.DataFrame() #Create new data frame to not numbers
numeric_features = ds.select_dtypes(include=[np.number]) #data frame with only numeric features
#numeric_features.drop('SalePrice',axis='columns', inplace=True)
numeric_features = numeric_features.columns #separate only index
ds_not_num = ds #copy ds to ds_not_num
print(ds_not_num.head())
ds_not_num.drop(numeric_features,axis='columns', inplace=True) #drop all numeric in ds_not_num
for i in ds_not_num.columns:
    a = ds[[i, 'SalePrice']].groupby(i).mean()
    #print(a['SalePrice'].sort_values(ascending=True))
    index = ds[[i]].groupby(i).mean().index #extract all results in column as index
    count = 1
    for z in index:
        ds[i].replace({z: count}, inplace = True)
        ds_test[i].replace({z: count}, inplace = True)
        count = count + 1